### 💡 核心痛点：为什么不用列表 (List)？

在 Python 里，`list` 其实是一个“偏心”的家伙：

  * **处理屁股（尾部）很快**：`append()` 和 `pop()` 都是 $O(1)$ 的神速。
  * **处理头部（开头）极慢**：如果你想在**最前面**插入一个数 (`insert(0, val)`) 或者删除第一个数 (`pop(0)`)，Python 必须把后面成千上万个元素**全部往后挪**或者**往前挪**一位。复杂度是 $O(N)$。

**后果：** 如果你在算法题（特别是 BFS 广度优先搜索）里用 `list` 当队列用，只要数据量一达，立马 **TLE (超时)**。

-----

# 📝 Python 算法笔记：双端队列 (Deque)

### 1\. 什么是 Deque？

  * **全称**：Double-Ended Queue。
  * **特点**：它是一种**两头都开口**的管子。
      * 你可以在**左边**进，也可以在**左边**出。
      * 你可以在**右边**进，也可以在**右边**出。
  * **必杀技**：无论是在**头部**还是**尾部**进行插入/删除，时间复杂度全是 **$O(1)$**（最快）。

### 2\. 代码怎么写？

它不在默认环境里，藏在一个叫 `collections` 的标准库里。

```python
from collections import deque  # 必须先导入！

# 1. 创建空队列
q = deque()  
# 或者把列表转成队列: q = deque([1, 2, 3])

# 2. 尾部操作 (和 list 一样)
q.append(10)    # 右边进
q.pop()         # 右边出

# 3. 头部操作 (Deque 的独门绝技)
q.appendleft(5) # 左边进 (插队！) --> 速度 O(1)
q.popleft()     # 左边出 (排队离开) --> 速度 O(1)



In [1]:
from collections import deque
q = deque()
q.append(10)
q.appendleft(5)
print(q)


deque([5, 10])


### 3. 更多常用操作

| 操作 | 说明 | 复杂度 |
|---|---|---|
| `q.append(x)` | 右边追加 | O(1) |
| `q.appendleft(x)` | 左边追加 | O(1) |
| `q.pop()` | 右边弹出 | O(1) |
| `q.popleft()` | 左边弹出 | O(1) |
| `q.extend([1,2,3])` | 右边批量追加 | O(k) |
| `q.extendleft([1,2,3])` | 左边批量追加（注意：顺序会反过来！） | O(k) |
| `q.rotate(n)` | 向右旋转 n 步（负数向左） | O(k) |
| `q[0]` / `q[-1]` | 查看队头/队尾（不弹出） | O(1) |
| `len(q)` | 队列长度 | O(1) |
| `q.clear()` | 清空队列 | O(1) |

### 4. 限制队列长度 (maxlen)

可以创建一个**固定大小**的队列，超出时自动丢弃另一端的元素：

```python
q = deque(maxlen=3)  # 最多放 3 个
q.append(1)  # [1]
q.append(2)  # [1, 2]
q.append(3)  # [1, 2, 3]
q.append(4)  # [2, 3, 4] ← 1 被自动挤掉了！
```

**应用场景**：保留"最近 N 条记录"，比如最近 N 次请求、滑动窗口等。

In [2]:
# 演示 maxlen 自动淘汰旧数据
from collections import deque

recent_logs = deque(maxlen=3)
recent_logs.append("登录成功")
recent_logs.append("查看订单")
recent_logs.append("修改密码")
print("当前日志:", list(recent_logs))

recent_logs.append("退出登录")  # 新的进来，最老的"登录成功"被挤掉
print("新增后:", list(recent_logs))

当前日志: ['登录成功', '查看订单', '修改密码']
新增后: ['查看订单', '修改密码', '退出登录']


### 5. 典型应用场景

| 场景 | 为什么用 Deque |
|---|---|
| **BFS 广度优先搜索** | 需要频繁 `popleft()`，list 会超时 |
| **滑动窗口** | 两端都要操作（加右边、删左边） |
| **最近 N 条记录** | 用 `maxlen` 自动淘汰旧数据 |
| **回文检测** | 两头同时比较，`popleft()` + `pop()` |
| **任务调度** | 可以从两端取任务（优先级队列简化版） |

### 6. Deque vs List 速度对比

| 操作 | `list` | `deque` |
|---|---|---|
| 尾部添加 `append()` | O(1) ✅ | O(1) ✅ |
| 尾部删除 `pop()` | O(1) ✅ | O(1) ✅ |
| **头部添加** `insert(0,x)` / `appendleft()` | **O(N) ❌** | O(1) ✅ |
| **头部删除** `pop(0)` / `popleft()` | **O(N) ❌** | O(1) ✅ |
| 随机访问 `[i]` | O(1) ✅ | O(N) ❌ |

> **结论**：如果你需要频繁操作**头部**，用 `deque`；如果需要频繁**随机访问**中间元素，用 `list`。

In [3]:
# 速度对比：头部删除 10000 次
import time
from collections import deque

n = 10000

# 测试 list
lst = list(range(n))
start = time.time()
while lst:
    lst.pop(0)  # 头部删除
list_time = time.time() - start

# 测试 deque
dq = deque(range(n))
start = time.time()
while dq:
    dq.popleft()  # 头部删除
deque_time = time.time() - start

print(f"list.pop(0)   耗时: {list_time:.4f} 秒")
print(f"deque.popleft() 耗时: {deque_time:.4f} 秒")
print(f"deque 快了约 {list_time / deque_time:.0f} 倍！")

list.pop(0)   耗时: 0.0045 秒
deque.popleft() 耗时: 0.0010 秒
deque 快了约 4 倍！


### 7. 实战示例：用 Deque 检测回文

回文是指正着读和反着读都一样的字符串，比如 `"racecar"`、`"上海自来水来自海上"`。

**思路**：把字符串放进 deque，每次同时从两头各取一个字符比较。

In [4]:
from collections import deque

def is_palindrome(s):
    """用双端队列检测回文"""
    dq = deque(s)
    
    while len(dq) > 1:  # 只要还剩 2 个以上字符
        left = dq.popleft()   # 取头
        right = dq.pop()      # 取尾
        if left != right:
            return False
    
    return True  # 全部匹配，是回文

# 测试
print(is_palindrome("racecar"))  # True
print(is_palindrome("hello"))    # False
print(is_palindrome("a"))        # True (单个字符)
print(is_palindrome(""))         # True (空字符串)

True
False
True
True


### 📌 总结口诀

> *   **两头开口选 Deque**
> *   **BFS 必用 popleft**
> *   **滑动窗口左删右加**
> *   **最近 N 条用 maxlen**